An additive hazards model is a statistical model used to analyze the time-to-event data, such as the survival of patients with a certain disease or the failure time of mechanical systems. It studies the risk factors which affect the survival time and their risk contributions, and also offer great flexibility due to its semi-parametric nature.

The hazard function for the survival time $T$ associated with a $p$-vector of possibly time-varying covariates $\mathbf{Z}(\cdot)$ takes the form 
$$\lambda(t\mid \mathbf{Z})=\lambda_0(t) + \boldsymbol{\beta}^{\prime}\mathbf{Z}(t)$$
under the  additive hazards model. 

For many reasons not all survival time are fullly observed, we only consider the right-censoring situation here and let $C$ be the censoring time. Denote the observed time by $X=\min\{T,C\}$ and the censoring indicator by $\delta=I(T\leq C)$. Define the observed-failure counting process $N(t)=I(X\leq t,\delta=1)$ which registers whether an uncensored failure has occurred by time $t$ and the corresponding at-risk indicator $Y(t)=I(X\geq t)$. Without loss of generality, denote 1 as the terminal time of observation and there are $n$ independent data $\{(X_i,\delta_i,\mathbf{Z}_i): i=1,2,\cdots,n\}$.

The least squares-type loss function of additive hazards model is 
$$L_1(\boldsymbol{\beta})=\frac{1}{2}\boldsymbol{\beta}^{\prime}V\boldsymbol{\beta}-\mathbf{b}^{\prime}\boldsymbol{\beta},$$
where $\overline{\mathbf{Z}}(t)=\sum_{j=1}^n Y_j(t) \mathbf{Z}_j(t) / \sum_{j=1}^n Y_j(t)$, $\mathbf{b}=\frac{1}{n}\sum_{i=1}^{n}\int_0^1 \mathbf{Z}_i(t)-\overline{\mathbf{Z}}(t) \mathrm{d} N_i(t)$ and $\mathbf{V}=\frac{1}{n} \sum_{i=1}^n \int_0^1 Y_i(t)\left\{\mathbf{Z}_i(t)-\overline{\mathbf{Z}}(t)\right\}^{\otimes 2} \mathrm{d} t$. We can estimate $\boldsymbol{\beta}$ by minimizing the loss function under sparsity constraint:
$$arg\min_{\beta \in R^p}L_1(\beta) , s.t.  || \beta ||_0 \leq s.$$

In [ ]:
# generate additive hazard model data
import numpy as np
import numpy.random as rd
import random

def make_ahad_data(n, coef_, c=0.5, rho=0):
    lam = 1
    p = coef_.shape[0]
    mu = np.zeros(p)
    Sigma = np.power(rho, np.abs(np.reshape(np.array([i-j for i in np.arange(1,p+1) for j in np.arange(1,p+1)]) ,newshape=(p,p))))
    x = rd.multivariate_normal(mu, Sigma, (10*n,))    # variables
    x = x[np.matmul(x,coef_)>-1,]
    x = x[0:n,]

    xcoef = np.matmul(x,coef_)
    #lam = np.maximum(1, np.ceil(np.max(-xcoef[xcoef<0])) )
    T = -np.log(rd.uniform(0,1, (n,)))/(lam + xcoef)    # survival time
    U = rd.uniform(0,c, (n,))           # censoring time
    delta = (T<=U)*1           # censoring indicator
    print("censoring rate:", np.sum(1-delta)/n )
    return x, np.minimum(T,U), delta

Here is Python code for solving sparse additive hazards model:  

In [2]:
import jax.numpy as jnp
from scope import ScopeSolver

n, p, k = 200, 10, 2
beta = np.zeros(p)
supportset = random.sample(range(0,p),k)
beta[supportset] = rd.uniform(2, 5, k)
x, y, delta = make_ahad_data(n,beta)

xsort = x[np.argsort(y),:]
ysort = y[np.argsort(y)]
deltasort = delta[np.argsort(y)]
ydiff = np.append(ysort[0],np.diff(ysort))
xbar = np.empty(shape=(n,p))
for i in range(0,n):
    xbar[i,:] = np.mean(xsort[i:n,:],axis=0)
b = np.matmul(deltasort, xsort-xbar)
V = np.empty(shape=(p,p))
#print(np.dot((x[1,:]-xbar[0:(1+1),:]).T,(x[1,:]-xbar[0:2,:])))
for i in range(n):
    V += ((xsort[i,:]-xbar[0:(i+1),:]).T) @ np.diag(ydiff[0:(i+1)]) @ (xsort[i,:]-xbar[0:(i+1),:])

def ahazard_objective(params):
    return jnp.dot(jnp.dot(V,params),params)/2 - jnp.dot(params,b) 

solver = ScopeSolver(p, k)
solver.solve(ahazard_objective, jit=True)
print("Estimated parameter:", solver.get_result()["params"], "objective:",solver.get_result()["value_of_objective"])
print("True parameter:", beta, "objective:",ahazard_objective(beta))

censoring rate: 0.56
Estimated parameter: [0.         0.         0.         0.         0.         0.
 0.         2.90210568 0.         2.37168955] objective: -84.62509155273438
True parameter: [0.        0.        0.        0.        0.        0.        0.
 2.5291107 0.        2.4299368] objective: -82.85272
